#**4.3 SGD를 사용한 MF 기본 알고리즘**

In [5]:
import os
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split 

base_src = 'drive/MyDrive/RecoSys/Data'
u_data_src = os.path.join(base_src,'u.data')
r_cols = ["user_id","movie_id",'rating','timestamp']
ratings = pd.read_csv(u_data_src,
                    sep = '\t',
                    names = r_cols,
                    encoding='latin-1')
#timestamp 제거
ratings = ratings[['user_id','movie_id','rating']].astype(int)

In [6]:
class MF(): 
  def __init__(self, ratings, hyper_params): 
    self.R = np.array(ratings)
    self.num_users, self.num_items = np.shape(self.R)
    self.K = hyper_params['K'] #key값 
    self.alpha = hyper_params['alpha'] 
    self.beta = hyper_params['beta'] #정규화 계수 
    self.iterations = hyper_params['iterations'] #반복 횟수
    self.verbose = hyper_params['verbose'] #학습과정 출력 여부 결정 

  def rmse(self):
    xs, ys = self.R.nonzero() #rating data에서 0이 아닌 요소의 인덱스
    self.predictions = []
    self.errors = [] 
    for x,y in zip(xs, ys): 
      prediction = self.get_prediction(x,y)
      self.predictions.append(prediction)
      self.errors.append(self.R[x,y]-prediction)
    self.predictions = np.array(self.predictions)
    self.errors = np.array(self.errors)

    return np.sqrt(np.mean(self.errors**2))

  def train(self): 
    self.P = np.random.normal(scale = 1./self.K,
                                size = (self.num_users,self.K)) #mean = 0(default) , scale : 표준편차 = 1/잠재변수의 개수
    self.Q = np.random.normal(scale = 1./self.K,
                                size = (self.num_items,self.K))
    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])

    rows, columns = self.R.nonzero()
    self.samples = [(i,j,self.R[i,j]) for i,j in zip(rows,columns)]

    training_process = []
    for i in range(self.iterations): 
      np.random.shuffle(self.samples)
      self.sgd() #sgd 실행 
      rmse = self.rmse() #새로운 rmse 계산 
      training_process.append((i+1, rmse)) #몇 번째의 rmse
      if self.verbose == True:
        if (i+1) % 10 == 0: #10번에 한 번씩 출력 
          print("Iteration : %d ; train RMSE = %.4f"%(i+1, rmse))
    return training_process 

  def get_prediction(self, i, j): 
    prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,].T) #전체 평점 + 유저에 대한 평가 경항 + 아이템에 대한 평가 경향 + 사용자 요인값*아이템 요인값
    return prediction 

  def sgd(self): 
    for i,j,r in self.samples:  #i,j : 인덱스, r : 평점 
      prediction = self.get_prediction(i,j)
      e = (r-prediction)
      
      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      self.b_d[j] += self.alpha * (e- (self.beta * self.b_d[j]))

      self.P[i,:] += self.alpha * ((e * self.Q[j,:] - self.beta * self.P[i,:]))
      self.Q[j,:] += self.alpha * ((e * self.P[i,:])- (self.beta * self.Q[j, :]))

R_temp = ratings.pivot(index = 'user_id',
                       columns = 'movie_id',
                       values = 'rating').fillna(0)
hyper_params = {
    'K' : 30,
    'alpha' : 0.001,
    'beta' : 0.02,
    'iterations' : 100,
    'verbose' : True
}

mf = MF(R_temp ,hyper_params)

train_process = mf.train()

Iteration : 10 ; train RMSE = 0.9585
Iteration : 20 ; train RMSE = 0.9373
Iteration : 30 ; train RMSE = 0.9280
Iteration : 40 ; train RMSE = 0.9224
Iteration : 50 ; train RMSE = 0.9181
Iteration : 60 ; train RMSE = 0.9138
Iteration : 70 ; train RMSE = 0.9085
Iteration : 80 ; train RMSE = 0.9013
Iteration : 90 ; train RMSE = 0.8914
Iteration : 100 ; train RMSE = 0.8793


#**4.4 train/test 분리 MF 알고리즘**

In [7]:
#shuffle 방식 사용 
#train/test set 분리 
from sklearn.utils import shuffle
train_size = 0.75
ratings = shuffle(ratings, random_state=2021)
cutoff = int(train_size * len(ratings)) 
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

class NEW_MF(): 
  def __init__(self, ratings, hyper_params): 
    self.R = np.array(ratings)
    #사용자 수(num_users)와 아이템 수(num_iterms)를 받아온다.
    self.num_users, self.num_items = np.shape(self.R)
    #아래는 MF weight 조절을 위한 하이퍼파라미터이다. 
    #K : 잠재요인의 수 
    self.K = hyper_params['K'] #key값 
    self.alpha = hyper_params['alpha'] #학습률
    self.beta = hyper_params['beta'] #정규화 계수 
    self.iterations = hyper_params['iterations'] #반복 횟수
    self.verbose = hyper_params['verbose'] #학습과정 출력 여부 결정 
  
    #데이터의 값이 연속값이 아닐 경우 발생하는 문제 해결 
    #itme id 
    item_id_index = []
    index_item_id = []
    for i, one_id in enumerate(ratings): #i : index, one_id : movie_id
      item_id_index.append([one_id, i])
      index_item_id.append([i, one_id])
    self.item_id_index = dict(item_id_index)
    self.index_item_id = dict(index_item_id)
    
    #user id 
    user_id_index = []
    index_user_id  = []
    for i, one_id in enumerate(ratings.T): #i : index, one_id : user_id
      user_id_index.append([one_id, i])    
      index_user_id.append([i, one_id])
    self.user_id_index = dict(user_id_index)
    self.index_user_id = dict(index_user_id)
  
  def rmse(self):
    #rating data에서 0이 아닌 요소의 인덱스
    xs, ys = self.R.nonzero() 
    #prediction과 error를 담을 리스트 변수 초기화 
    self.predictions = []
    self.errors = [] 
    #평점이 있는 요소(사용자 x, 아이템 y) 각각에 대해서 아래의 코드를 실행한다. 
    for x,y in zip(xs, ys): 
      #사용자 x, 아이템 y에 대해서 평점 예측치를 get_predition() 함수를 사용해서 계산한다.
      prediction = self.get_prediction(x,y)
      #예측값을 예측값 리스트에 추가한다.
      self.predictions.append(prediction)
      #실제값(R)과 예측값의 차이(errors) 계산해서 오차값 리스트에 추가한다.
      self.errors.append(self.R[x,y]-prediction)
    #예측값 리스트와 오차값 리스트를 numpy array형태로 변환한다.
    self.predictions = np.array(self.predictions)
    #error를 활용해서 RMSE 도출 
    self.errors = np.array(self.errors)

    return np.sqrt(np.mean(self.errors**2))

  def get_prediction(self, i, j): 
    #사용자 i, 아이템 j에 대한 평점 예측치를 앞에서 배웠던 식을 이용해서 구한다.
    prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,].T) #전체 평점 + 유저에 대한 평가 경항 + 아이템에 대한 평가 경향 + 사용자 요인값*아이템 요인값
    return prediction 

  def sgd(self): 
    for i,j,r in self.samples:  #i,j : 인덱스, r : 평점 
      #사용자 i, 아이템 j에 대한 평점 예측치 계산 
      prediction = self.get_prediction(i,j)
      #실제 평점과 비교한 오차 계산 
      e = (r-prediction)
      
      #사용자 평가 경향 계산 및 업데이트
      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      #아이템 평가 경향 계산 및 업데이트
      self.b_d[j] += self.alpha * (e- (self.beta * self.b_d[j]))

      #P 행렬 계산 및 업데이트
      self.P[i,:] += self.alpha * ((e * self.Q[j,:] - self.beta * self.P[i,:]))
      #Q 행렬 계산 및 업데이트
      self.Q[j,:] += self.alpha * ((e * self.P[i,:])- (self.beta * self.Q[j, :]))

  #Test set 선정 
  def set_test(self, ratings_test):
    test_set = []
    for i in range(len(ratings_test)):
      x = self.user_id_index[ratings_test.iloc[i,0]] #user index
      y = self.item_id_index[ratings_test.iloc[i,1]] #item index
      z = ratings_test.iloc[i,2] #실제 평점 
      test_set.append([x,y,z])
      self.R[x,y] = 0 
    self.test_set = test_set 
    return test_set 

  #Test set RMSE 계산 
  def test_rmse(self):
    error = 0
    for one_set in self.test_set: 
      predicted = self.get_prediction(one_set[0],one_set[1])
      #pow : 차승 
      error += pow(one_set[2]-predicted, 2)
    return np.sqrt(error/len(self.test_set))

  def test(self):
    self.P = np.random.normal(scale=1./self.K,
                              size = (self.num_users, self.K))
    self.Q = np.random.normal(scale=1./self.K,
                              size = (self.num_items, self.K))
    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])

    rows, columns = self.R.nonzero() 
    self.samples = [(i,j,self.R[i,j]) for i,j in zip(rows, columns)]

    training_process = []
    for i in range(self.iterations):
      np.random.shuffle(self.samples)
      self.sgd()
      rmse1 = self.rmse() #training set
      rmse2 = self.test_rmse() #test set
      training_process.append((i+1, rmse1, rmse2))
      if self.verbose == True:
        if (i+1) % 10 == 0:
          print("Iteration : %d ; train RMSE = %.4f; test RMSE %.4f"%(i+1, rmse1, rmse2))
    return training_process 

  def get_one_prediction(self, user_id, item_id):
    return self.get_prediction(self.user_id_index[user_id],
                               self.item_id_index[item_id])
  
  def full_prediction(self):
    return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)
  
R_temp = ratings.pivot(index = 'user_id',
                       columns = 'movie_id',
                       values = 'rating').fillna(0)
      
hyper_params = {
    'K' : 30,
    'alpha':0.001,
    'beta':0.02,
    'iterations':100,
    'verbose':True
}

mf = NEW_MF(R_temp, hyper_params)
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration : 10 ; train RMSE = 0.9666; test RMSE 0.9808
Iteration : 20 ; train RMSE = 0.9412; test RMSE 0.9624
Iteration : 30 ; train RMSE = 0.9298; test RMSE 0.9553
Iteration : 40 ; train RMSE = 0.9229; test RMSE 0.9517
Iteration : 50 ; train RMSE = 0.9180; test RMSE 0.9495
Iteration : 60 ; train RMSE = 0.9141; test RMSE 0.9480
Iteration : 70 ; train RMSE = 0.9103; test RMSE 0.9469
Iteration : 80 ; train RMSE = 0.9062; test RMSE 0.9458
Iteration : 90 ; train RMSE = 0.9012; test RMSE 0.9447
Iteration : 100 ; train RMSE = 0.8948; test RMSE 0.9432


In [8]:
print(mf.full_prediction()) #전체 예측 

[[3.91680053 3.41630481 2.99347295 ... 3.37707931 3.48997105 3.45824319]
 [3.80364857 3.28775338 2.89778476 ... 3.25584669 3.38173453 3.35013471]
 [3.38064047 2.87533035 2.48248947 ... 2.87353535 2.97752542 2.96088126]
 ...
 [4.16070048 3.60464936 3.21176061 ... 3.58588075 3.71160025 3.68693652]
 [4.33029824 3.78634515 3.38294593 ... 3.74848678 3.87043714 3.84495776]
 [3.84971015 3.32109453 2.92597107 ... 3.28000187 3.42266809 3.39469067]]


In [9]:
print(mf.get_one_prediction(1,2)) #사용자 1번의 2번 아이템에 대한 예측

3.416304809264164


#**4.5 MF의 최적 파라미터 찾기**

In [10]:
#최적의 K 값 찾기 
results = [] #성능에 대한 결과값
index = [] #k값을 갖는 인덱스 
  
R_temp = ratings.pivot(index = 'user_id',
                       columns = 'movie_id',
                       values = 'rating').fillna(0)

for K in range(50,61,10):
  print(f'K : {K}')
  hyper_params = {
      'K' : K,
      'alpha' : 0.001,
      'beta' : 0.02,
      'iterations' : 300,
      'verbose' : True
  }
  mf = NEW_MF(R_temp,
              hyper_params)
  test_set = mf.set_test(ratings_test)
  result = mf.test() 
  index.append(K)
  results.append(result)

K : 50
Iteration : 10 ; train RMSE = 0.9669; test RMSE 0.9807
Iteration : 20 ; train RMSE = 0.9417; test RMSE 0.9623
Iteration : 30 ; train RMSE = 0.9305; test RMSE 0.9552
Iteration : 40 ; train RMSE = 0.9239; test RMSE 0.9515
Iteration : 50 ; train RMSE = 0.9195; test RMSE 0.9493
Iteration : 60 ; train RMSE = 0.9160; test RMSE 0.9479
Iteration : 70 ; train RMSE = 0.9130; test RMSE 0.9468
Iteration : 80 ; train RMSE = 0.9099; test RMSE 0.9459
Iteration : 90 ; train RMSE = 0.9063; test RMSE 0.9449
Iteration : 100 ; train RMSE = 0.9018; test RMSE 0.9437
Iteration : 110 ; train RMSE = 0.8956; test RMSE 0.9419
Iteration : 120 ; train RMSE = 0.8873; test RMSE 0.9395
Iteration : 130 ; train RMSE = 0.8765; test RMSE 0.9363
Iteration : 140 ; train RMSE = 0.8633; test RMSE 0.9327
Iteration : 150 ; train RMSE = 0.8481; test RMSE 0.9292
Iteration : 160 ; train RMSE = 0.8316; test RMSE 0.9260
Iteration : 170 ; train RMSE = 0.8139; test RMSE 0.9234
Iteration : 180 ; train RMSE = 0.7950; test RMSE 0

In [11]:
summary = []
for i in range(len(results)):
  RMSE = [] #test RMSE 
  for result in results[i]:
    RMSE.append(result[2]) #result[0] : index, result[1] : train RMSE, result[2] : test RMSE 
  min = np.min(RMSE)
  j = RMSE.index(min)
  summary.append([index[i], j+1, RMSE[j]]) #K값, iteration 값, 최소의 RMSE 값 

In [12]:
summary

[[50, 210, 0.9184930243334138], [60, 220, 0.9157432593437194]]